> Note: structure of GraphQL response is not consistent, these examples may not work in all cases.

In [1]:
# !pip install twitter-api-client==0.9.0

In [ ]:
from twitter.scraper import Scraper
from twitter.util import *
import pandas as pd

scraper = Scraper(session=init_session())

# example tweet
tweet = scraper.tweets_details([1476988122986647553], limit=500, pbar=False)

# unnest items and filter deleted tweets
items = [y for x in find_key(tweet, 'items') for y in x if not find_key(y, 'tombstone')]

# index into relevant data points
tweet_results = [x.get('result') for x in find_key(items, 'tweet_results')]

df = (
    pd.json_normalize(tweet_results)
    # remove duplicate replies if needed
    .drop_duplicates('rest_id')
    # clean up column names for illustrative purposes
    .assign(date=lambda x: pd.to_datetime(x['legacy.created_at']).dt.strftime("%Y-%m-%d %H:%M:%S"))
    .assign(root_tweet=lambda x: x['legacy.conversation_id_str'])
    .assign(text=lambda x: x['legacy.full_text'])
    .assign(tweet=lambda x: x['rest_id'])
    .assign(username=lambda x: x['core.user_results.result.legacy.screen_name'])
    # sort by newest replies to root_tweet
    .sort_values('date', ascending=False)
    .reset_index(drop=True)
)

In [3]:
df[[
    'date',
    'username',
    'tweet',
    'text',
    'root_tweet',
]]

,date,username,tweet,text,root_tweet
0,2023-05-25 19:52:44,dtweedys,1661822628393254913,@MKBHD 🤺🔸💜 #SubTweets ftw 🔸🌑🕊️,1476988122986647553
1,2023-05-07 14:30:30,BallSchlonger,1655218553858015233,@jacknft_4 @MKBHD sorry i cant,1476988122986647553
2,2023-05-03 21:13:33,jacknft_4,1653870434075410436,@MKBHD Great! Dm me please,1476988122986647553
3,2023-05-02 17:38:40,tanmayshah28,1653453969476456450,@MKBHD Thank you! Lets too!,1476988122986647553
4,2023-04-02 14:08:55,iStevenPlays,1642529545415712768,@MKBHD I got you.\n\n‘🔥__🔥’,1476988122986647553
...,...,...,...,...,...
199,2021-12-31 18:46:59,sbeams,1476988269187448832,@MKBHD https://t.co/1Fu5cKmPTn,1476988122986647553
200,2021-12-31 18:46:48,woworeoo,1476988222609793033,@MKBHD Yeah Yeah no more tweets from you,1476988122986647553
201,2021-12-31 18:46:46,disisjorj,1476988217576640517,@MKBHD @vassizzle,1476988122986647553
202,2021-12-31 18:46:46,TechWizYT,1476988215043276802,@MKBHD Great advice!,1476988122986647553
